In [ ]:
# -*- coding: utf-8 -*-
"""
Example script to serve perform the SPECT simulation. This is the equivalent of
    ./run_simulation_SPECT.sh

Author: Kris Thielemans
"""

# Initial imports

In [ ]:
import stir
from stirextra import *
import os
from distutils.dir_util import copy_tree

# go to directory with input files

In [ ]:
# adapt this path to your situation (or start everything in the exercises directory)
os.chdir(os.getenv('STIR_exercises_PATH'))

# make directory for output files

In [ ]:
os.makedirs('working_folder/single_slice_SPECT', exist_ok=True)
copy_tree('EX_simulation_single_slice_SPECT/', 'working_folder/single_slice_SPECT')
os.chdir('working_folder/single_slice_SPECT')

# read in data

In [ ]:
emission=stir.FloatVoxelsOnCartesianGrid.read_from_file('emission.hv')
atn=stir.FloatVoxelsOnCartesianGrid.read_from_file('CTAC.hv')
template=stir.ProjData.read_from_file('template_sinogram.hs')

# Create projection matrix and projector

In [ ]:
pm=stir.ProjMatrixByBinSPECTUB()

See how we can set parameters for PSF.

In [ ]:
help(pm.set_resolution_model)

We will use 2D-PSF for speed

In [ ]:
pm.set_resolution_model(0.0163, 0.1466, False)

In [ ]:
pm.set_attenuation_image_sptr(atn)

In [ ]:
projectors=stir.ProjectorByBinPairUsingProjMatrixByBin(pm)
projectors.get_forward_projector().set_up(template.get_proj_data_info(), emission)

# Create output data object, and forward project

In [ ]:
proj_data=stir.ProjDataInMemory(template.get_exam_info(), template.get_proj_data_info())

In [ ]:
projectors.get_forward_projector().forward_project(proj_data, emission)

# Write to file for evaluation exercise

In [ ]:
proj_data.write_to_file('my_sim.hs');

## Add noise to the data.

Up to STIR 5.1, the Poisson random noise simulator needs to be
called from the command line.

!poisson_noise my_noisy_data.hs my_sim.hs 1 1

In more recent STIR, this can be done from Python. As always, use `help` to find out what the parameters mean.

In [ ]:
poisson_generator = stir.GeneralisedPoissonNoiseGenerator(1, True)

In [ ]:
noisy_proj_data = stir.ProjDataInMemory(template.get_exam_info(), template.get_proj_data_info())
poisson_generator.generate_random(noisy_proj_data, proj_data)
noisy_proj_data.write_to_file('my_noisy_data.hs')

# What to do now?

Open the `evaluate_simulate_SPECT` notebook to read and display this data.